# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('customer_churn').getOrCreate()

In [4]:
df = spark.read.csv('customer_churn.csv', header=True, inferSchema=True)

In [5]:
df.show()

+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|              Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+-------------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|   Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|      Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|        Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|      Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|     

In [7]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [8]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [22]:
from pyspark.sql.functions import countDistinct, year, current_date

In [32]:
df.select(df['Onboard_date']-current_date()).printSchema()

root
 |-- (Onboard_date - current_date()): calendarinterval (nullable = true)



In [33]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [34]:
my_col = ['Age','Total_Purchase','Years','Num_Sites']

In [36]:
df.select(my_col).describe().show()

+-------+-----------------+-----------------+-----------------+------------------+
|summary|              Age|   Total_Purchase|            Years|         Num_Sites|
+-------+-----------------+-----------------+-----------------+------------------+
|  count|              900|              900|              900|               900|
|   mean|41.81666666666667|10062.82403333334| 5.27315555555555| 8.587777777777777|
| stddev|6.127560416916251|2408.644531858096|1.274449013194616|1.7648355920350969|
|    min|             22.0|            100.0|              1.0|               3.0|
|    max|             65.0|         18026.01|             9.15|              14.0|
+-------+-----------------+-----------------+-----------------+------------------+



In [39]:
from pyspark.ml.feature import VectorAssembler,VectorIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [47]:
train , test = df.randomSplit([0.7,0.3])

In [40]:
log_reg = LogisticRegression(featuresCol='features', labelCol='Churn')

In [41]:
assembler = VectorAssembler(inputCols=my_col, outputCol='features')

In [42]:
pipline = Pipeline(stages=[assembler,log_reg])

In [48]:
fit_model = pipline.fit(train)

In [50]:
results = fit_model.transform(test)

In [51]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')
auc = my_eval.evaluate(results)
auc

0.7963155163466529

In [44]:
test_data = spark.read.csv('new_customers.csv', header=True, inferSchema=True)

In [54]:
results2 = fit_model.transform(test_data)

In [56]:
results2.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+------------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|         Company|        prediction|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+------------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                  6|                   6|               6|                 6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|               null|                null|            null|0.6666666666666666|
| stddev|         null| 15.715

In [57]:
results2.select(['Company', 'prediction']).show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [53]:
test_data.describe().show()

+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|summary|        Names|               Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|         Company|
+-------+-------------+------------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+----------------+
|  count|            6|                 6|                6|                 6|                6|                 6|                  6|                   6|               6|
|   mean|         null|35.166666666666664|7607.156666666667|0.8333333333333334|6.808333333333334|12.333333333333334|               null|                null|            null|
| stddev|         null| 15.71517313511584|4346.008232825459| 0.408248290463863|3.708737880555414|3.3862466931200785|         

In [12]:
df.select(countDistinct('Location'),countDistinct('Company')).show()

+------------------------+-----------------------+
|count(DISTINCT Location)|count(DISTINCT Company)|
+------------------------+-----------------------+
|                     900|                    873|
+------------------------+-----------------------+

